In [3]:
import pandas as pd
from dataclasses import dataclass, field

# An algebraic backtesting framework

## The position type

The core of a general event-driven backtesting framework is either the *Position* or *Order* object, depending on timescales of interest, among other things. We will abstract this notion and define an abstract category of *Positions* and provide a minimal structure with which we can emulate trading activity. We begin first with a concrete implementation of the base *Position* type:

In [4]:
pdts = pd.Timestamp
@dataclass
class Position:
    ticker_symbol: str
    init_date: str or None
    shares: float
    roll_date : None = None
    id: int = field(default=-1, repr=False)
    def __post_init__(self):
        if not pdts(self.init_date).dayofweek < 5:
            raise Exception(f'{self.init_date} is a weekend')
    @property
    def is_open(self):
        if self.shares == 0:
            return False
        else:
            return True
    def __str__(self):
        return f"Position(Ticker: {self.ticker_symbol}, Shares: {self.shares}, "\
                   f"InitDate: {self.init_date}, RollDate: {self.roll_date})"
    def __bool__(self):
        return self.is_open
    def __hash__(self):
        self.id = hash((self.ticker_symbol, self.init_date))
        return self.id

Objects belonging to this dataclass are defined by a symbol and an entry date, as far as the ``__hash__`` method is concerned. Though abstractly, the category of *Positions* contain the tuple (symbol, entry date, size, roll date) as objects. Morphisms are defined as updates of the latter two tuple entries obeying some obvious causality and conservation rules. In fact, what is natural is to give this category a monoidal structure defined by the binary